In [5]:
import pandas as pd

# Attempt to load the data using UTF-16 encoding, which may resolve encoding issues
variables_df = pd.read_csv('../data/variables_res-soft-class_2024-10-31_21-58.csv', delimiter=';', encoding='utf-16')
data_df = pd.read_csv('../data/data_res-soft-class_2024-10-31_21-46.csv', delimiter=';', encoding='utf-16')
values_df = pd.read_csv('../data/values_res-soft-class_2024-10-31_21-58.csv', delimiter=';', encoding='utf-16')

# Displaying the first few rows of each DataFrame to verify successful loading
variables_df.head() 
 

,VAR,LABEL,TYPE,INPUT,QUESTION
0,CASE,Interview-Nummer (fortlaufend),METRIC,SYSTEM,NaN
1,SERIAL,Personenkennung oder Teilnahmecode (sofern ver...,TEXT,SYSTEM,NaN
2,REF,Referenz (sofern im Link angegeben),TEXT,SYSTEM,NaN
3,QUESTNNR,"Fragebogen, der im Interview verwendet wurde",TEXT,SYSTEM,NaN
4,MODE,Interview-Modus,TEXT,SYSTEM,NaN


In [4]:
data_df.head()

,CASE,SERIAL,REF,QUESTNNR,MODE,STARTED,R001_CN,R001x01,R001x11,R001x12,...,TIME_SUM,MAILSENT,LASTDATA,FINISHED,Q_VIEWER,LASTPAGE,MAXPAGE,MISSING,MISSREL,TIME_RSI
0,63,NaN,NaN,base,interview,2024-10-18 08:46:14,NaN,NaN,NaN,NaN,...,10,NaN,2024-10-18 08:46:24,0,0,1,1,0,0,"0,6"
1,68,NaN,NaN,base,interview,2024-10-18 09:30:17,NaN,NaN,NaN,NaN,...,174,NaN,2024-10-21 09:14:27,0,0,2,2,0,0,"0,38"
2,69,NaN,NaN,base,interview,2024-10-18 10:01:06,NaN,NaN,NaN,NaN,...,6,NaN,2024-10-18 10:01:12,0,0,1,1,0,0,1
3,70,NaN,NaN,base,interview,2024-10-18 10:02:52,1.0,2.0,2.0,1.0,...,632,NaN,2024-10-18 10:13:24,1,0,2,2,0,0,"1,13"
4,72,NaN,NaN,base,interview,2024-10-18 11:52:05,1.0,2.0,2.0,1.0,...,162,NaN,2024-10-18 11:54:47,1,0,2,2,0,0,"0,96"


In [2]:

values_df.head()


(        VAR                                              LABEL    TYPE  \
 0      CASE                     Interview-Nummer (fortlaufend)  METRIC   
 1    SERIAL  Personenkennung oder Teilnahmecode (sofern ver...    TEXT   
 2       REF                Referenz (sofern im Link angegeben)    TEXT   
 3  QUESTNNR       Fragebogen, der im Interview verwendet wurde    TEXT   
 4      MODE                                    Interview-Modus    TEXT   
 
     INPUT QUESTION  
 0  SYSTEM      NaN  
 1  SYSTEM      NaN  
 2  SYSTEM      NaN  
 3  SYSTEM      NaN  
 4  SYSTEM      NaN  ,
    CASE  SERIAL  REF QUESTNNR       MODE              STARTED  R001_CN  \
 0    63     NaN  NaN     base  interview  2024-10-18 08:46:14      NaN   
 1    68     NaN  NaN     base  interview  2024-10-18 09:30:17      NaN   
 2    69     NaN  NaN     base  interview  2024-10-18 10:01:06      NaN   
 3    70     NaN  NaN     base  interview  2024-10-18 10:02:52      1.0   
 4    72     NaN  NaN     base  intervie

In [6]:
# Filtering for relevant information: ID, DOI, and category responses marked as "2" (selected)

# Extract mappings from variables and values for category labels
variables_labels = variables_df[['VAR', 'LABEL']].set_index('VAR').to_dict()['LABEL']
values_selected = values_df[values_df['RESPONSE'] == 2][['VAR', 'MEANING']]

# Merge the information to get category label names for selected responses
selected_responses = data_df[['CASE', 'REF']].copy()  # Starting with CASE and REF columns

selected_responses

,CASE,REF
0,63,NaN
1,68,NaN
2,69,NaN
3,70,NaN
4,72,NaN
...,...,...
260,405,NaN
261,407,NaN
262,408,NaN
263,409,NaN


In [8]:

# Identify the columns corresponding to categories in data_df that match selected values in "values_df"
for var in values_selected['VAR'].unique():
    if var in data_df.columns:
        selected_responses[var] = data_df[var]  # Add each relevant category column

selected_responses

,CASE,REF,R001x01,R001x11,R001x12,R001x13,R001x14,R001x15,R001x02,R001x21,...,R001x32,R001x33,R001x34,R001x35,R001x36,R001x37,R001x38,R001x99,R003,R005
0,63,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN
1,68,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8,NaN
2,69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,NaN
3,70,NaN,2.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,3,NaN
4,72,NaN,2.0,2.0,1.0,1.0,1.0,1.0,1.0,NaN,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,9,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
260,405,NaN,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,13,1.0
261,407,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13,NaN
262,408,NaN,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1,1.0
263,409,NaN,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1,1.0


In [9]:

# Filter to include only rows where categories have a value of "2" (selected)
filtered_responses = selected_responses.melt(id_vars=['CASE', 'REF'], var_name='Category', value_name='Response')
filtered_responses = filtered_responses[filtered_responses['Response'] == 2]

filtered_responses

,CASE,REF,Category,Response
3,70,NaN,R001x01,2.0
4,72,NaN,R001x01,2.0
5,73,NaN,R001x01,2.0
12,84,NaN,R001x01,2.0
13,88,NaN,R001x01,2.0
...,...,...,...,...
5646,178,NaN,R005,2.0
5648,180,NaN,R005,2.0
5710,270,NaN,R005,2.0
5712,272,NaN,R005,2.0


In [7]:


# Map category codes to their labels and clean up the output
filtered_responses['Category_Label'] = filtered_responses['Category'].map(variables_labels)
filtered_responses.drop(columns='Response', inplace=True)

filtered_responses
# Display the result
# import ace_tools as tools; 
# tools.display_dataframe_to_user(name="Selected Votes Summary", dataframe=filtered_responses)


,CASE,REF,Category,Category_Label
3,70,NaN,R001x01,"Category: Modelling, Simulation and Data Analy..."
4,72,NaN,R001x01,"Category: Modelling, Simulation and Data Analy..."
5,73,NaN,R001x01,"Category: Modelling, Simulation and Data Analy..."
12,84,NaN,R001x01,"Category: Modelling, Simulation and Data Analy..."
13,88,NaN,R001x01,"Category: Modelling, Simulation and Data Analy..."
...,...,...,...,...
5646,178,NaN,R005,New_Category
5648,180,NaN,R005,New_Category
5710,270,NaN,R005,New_Category
5712,272,NaN,R005,New_Category


In [12]:
category_columns = [col for col in data_df.columns if col.startswith('R001x')]



# Initialize a result dataframe to store each CASE, REF, and a list of categories marked as "2"

votes_summary = []



# Iterate over each row in the data to gather CASE, REF, and list of selected categories

for _, row in data_df.iterrows():

    case = row['CASE']

    ref = row['REF']

    selected_categories = []



    # Check each category column if it is marked with "2"

    for col in category_columns:

        if row[col] == 2:

            category_label = variables_labels.get(col, col)  # Get label if available, or fallback to column name

            selected_categories.append(category_label)



    # Append only if there are selected categories

    if selected_categories:

        votes_summary.append({

            'CASE': case,

            'REF': ref,

            'Selected_Categories': selected_categories

        })



# Convert the summary list into a DataFrame for easier display

votes_summary_df = pd.DataFrame(votes_summary)
votes_summary_df['DOI'] = data_df.set_index('CASE').loc[votes_summary_df['CASE'], 'R004_01'].values
votes_summary_df['Voter'] = data_df.set_index('CASE').loc[votes_summary_df['CASE'], 'R003'].values


votes_summary_df

,CASE,REF,Selected_Categories,DOI,Voter
0,70,NaN,"[Category: Modelling, Simulation and Data Anal...",10.21105/joss.07134,3
1,72,NaN,"[Category: Modelling, Simulation and Data Anal...",10.21105/joss.07134,9
2,73,NaN,"[Category: Modelling, Simulation and Data Anal...",10.21105/joss.06914,9
3,74,NaN,"[Category: Research Infrastructure Software, C...",10.21105/joss.06914,2
4,75,NaN,[Category: Technology Research Software],10.21105/joss.06825,2
...,...,...,...,...,...
238,399,NaN,"[Category: Modelling, Simulation and Data Anal...",10.21105/joss.02825,13
239,404,NaN,"[Category: Technology Research Software, Categ...",10.21105/joss.03465,13
240,405,NaN,"[Category: Modelling, Simulation and Data Anal...",10.21105/joss.03097,13
241,408,NaN,"[Category: Modelling, Simulation and Data Anal...",10.21105/joss.03097,1


In [13]:
voter_5_votes = votes_summary_df[votes_summary_df['Voter'] == 5]

voter_5_votes

,CASE,REF,Selected_Categories,DOI,Voter
231,389,NaN,"[Category: Modelling, Simulation and Data Anal...",10.21105/joss.06203,5
232,391,NaN,"[Category: Modelling, Simulation and Data Anal...",10.21105/joss.06358,5
233,393,NaN,"[Category: Modelling, Simulation and Data Anal...",10.21105/joss.05855,5
235,396,NaN,"[Category: Modelling, Simulation and Data Anal...",10.21105/joss.02017,5
236,397,NaN,"[Category: Modelling, Simulation and Data Anal...",10.21105/joss.05940,5
237,398,NaN,"[Category: Modelling, Simulation and Data Anal...",10.21105/joss.02825,5
